In [52]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from itertools import product
from transformers import BertTokenizer, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from projAll import filtered_labels_at_least_5_list, CustomBertModel, create_dataset, train_with_validation, test, combined_df

In [53]:
param_grid = {
    'learning_rate': [5e-5, 3e-5, 1e-5],
    'batch_size': [16, 32],
    'num_epochs': [4, 6]
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
label_mapping = {label: idx for idx, label in enumerate(filtered_labels_at_least_5_list)}

In [54]:
# df = pd.read_csv('csv/merged_aug.csv')
df = combined_df

In [55]:
train_val_df, test_df = train_test_split(df,
                                         test_size=0.2,
                                         stratify=df['Artifact Id'],
                                         random_state=42)
train_df, val_df = train_test_split(train_val_df,
                                         test_size=0.2,
                                         stratify=train_val_df['Artifact Id'],
                                         random_state=42)

In [56]:
def plot(acc_train, acc_val, acc_test, f1_train, f1_val, f1_test, params):
    num_epochs = params['num_epochs']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    
    epochs = range(1, num_epochs + 1)

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))  

    # Plot Accuracy
    axes[0].plot(epochs, acc_train, color='blue', linestyle='-', label='Train Accuracy')
    axes[0].plot(epochs, acc_val, color='red', linestyle='-', label='Validation Accuracy')
    axes[0].set_title('Accuracy Over Epochs')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Accuracy (%)')
    axes[0].legend()
    axes[0].grid(True)

    # Plot F1 Score
    axes[1].plot(epochs, f1_train, color='blue', linestyle='-', label='Train F1 Score')
    axes[1].plot(epochs, f1_val, color='red', linestyle='-', label='Validation F1 Score')
    axes[1].set_title('F1 Score Over Epochs')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('F1 Score')
    axes[1].legend()
    axes[1].grid(True)

    test_results_text = f"Test Accuracy: {acc_test:.2f}%\nTest F1 Score: {f1_test:.4f}"
    props = dict(boxstyle='round,pad=0.5', facecolor='lightgray', edgecolor='black')
    fig.text(0.5, -0.05, test_results_text, fontsize=10, bbox=props, ha='center')

    fig.suptitle(f"Training, Validation, and Test Metrics\n"
                 f"Epochs: {num_epochs}, Learning Rate: {learning_rate}, Batch Size: {batch_size}", fontsize=12)

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for suptitle
    plt.show()

In [59]:
def grid_search(param_grid):
    best_f1 = 0
    best_params = {}
    
    grid_combinations = list(product(*param_grid.values()))
    
    train_dataset = create_dataset(train_df, tokenizer, label_mapping)
    val_dataset = create_dataset(val_df, tokenizer, label_mapping)
    test_dataset = create_dataset(test_df, tokenizer, label_mapping)
    
    
    for params in grid_combinations:
        current_params = dict(zip(param_grid.keys(), params))
        
        print(f"Training with params: {current_params}")
        
        learning_rate = current_params['learning_rate']
        batch_size = current_params['batch_size']
        num_epochs = current_params['num_epochs']
        
        model = CustomBertModel(num_labels=len(filtered_labels_at_least_5_list))
        optimizer = AdamW(model.parameters(), lr=learning_rate)
        model.to(device)

        train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
        val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)
        test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

        f1_train, f1_val, acc_train, acc_val = train_with_validation(model, train_loader, val_loader, optimizer, device, num_epochs)

        predictions, true_labels = test(model, test_loader, device)
        
        f1_test = f1_score(true_labels, predictions, average='weighted')
        acc_test = accuracy_score(true_labels, predictions) * 100

        plot(acc_train, acc_val, acc_test, f1_train, f1_val, f1_test, current_params)
        
        if f1_test > best_f1:
            best_f1 = f1_test
            best_params = current_params
    
    print(f"Best F1 Score: {best_f1:.4f} with parameters: {best_params}")
    return best_params
    # return acc_train, acc_val, acc_test, f1_train, f1_val, f1_test, best_params

params = grid_search(param_grid)
# acc_train, acc_val, acc_test, f1_train, f1_val, f1_test, params = grid_search(param_grid)


Training with params: {'learning_rate': 5e-05, 'batch_size': 16, 'num_epochs': 4}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Epoch 1/4:   0%|          | 0/4 [00:00<?, ?it/s]